In [1]:
import numpy as np
import pandas as pd
import scipy as sp

import os
import glob

from nfl import Team, Game, Play, Player

In [2]:
games = pd.read_csv('data/games.csv')

In [3]:
players = pd.read_csv('data/players.csv')

In [4]:
plays = pd.read_csv('data/plays.csv')

In [5]:
weeks = {}
for i in range(1):
    name = f'week{i+1}'
    weeks[name] = pd.read_csv(os.path.join('data',name+'.csv'))

In [6]:
team_names = games['homeTeamAbbr'].unique()

In [7]:
teams = {}
for name in team_names:
    teams[name] = Team(abbr=name)

In [8]:
for i in range(games.shape[0]):
    _game_info = games.loc[i]
    gameId = _game_info['gameId']
    homeTeamAbbr = _game_info['homeTeamAbbr']
    visitorTeamAbbr = _game_info['visitorTeamAbbr']
    week = _game_info['week']
    
    game_plays = plays[plays['gameId'] == gameId]
    home_game_plays = game_plays[game_plays['possessionTeam']==homeTeamAbbr]
    away_game_plays = game_plays[game_plays['possessionTeam']==visitorTeamAbbr]
    
    teams[homeTeamAbbr].games[f'week{week}'] = Game(gameId,opponent=visitorTeamAbbr,game_info=_game_info,
                                                    play_data=away_game_plays,location='home')
    teams[visitorTeamAbbr].games[f'week{week}'] = Game(gameId,opponent=homeTeamAbbr,game_info=_game_info,
                                                    play_data=home_game_plays,location='away')

In [9]:
team = teams['IND']

In [10]:
team.games

{'week1': <nfl.Game at 0x12615aa20>,
 'week2': <nfl.Game at 0x107814400>,
 'week3': <nfl.Game at 0x1078365f8>,
 'week4': <nfl.Game at 0x1199a1a58>,
 'week5': <nfl.Game at 0x1199bf828>,
 'week6': <nfl.Game at 0x1199edb38>,
 'week7': <nfl.Game at 0x1199fbc18>,
 'week8': <nfl.Game at 0x119a29dd8>,
 'week10': <nfl.Game at 0x119a55dd8>,
 'week11': <nfl.Game at 0x119ab3cf8>,
 'week12': <nfl.Game at 0x119acec18>,
 'week13': <nfl.Game at 0x119af8978>,
 'week14': <nfl.Game at 0x119b11668>,
 'week15': <nfl.Game at 0x119b2c908>,
 'week16': <nfl.Game at 0x119b3bac8>,
 'week17': <nfl.Game at 0x119bf0358>}

In [11]:
for key in ('week1',):
    game = team.games[key]
    
    week_data = weeks[key]
    
    for i in game.play_data.index:
        play = game.play_data.loc[i]
        player_tracking = week_data[(week_data['gameId']==play['gameId'])&(week_data['playId']==play['playId'])]
        
        game.plays.append(Play(play['playId'],play_data=play,player_tracking=player_tracking))

In [12]:
game.plays[0].play_data

gameId                                                           2018090902
playId                                                                   81
playDescription           (14:11) (Shotgun) A.Dalton pass short left int...
quarter                                                                   1
down                                                                      2
yardsToGo                                                                 3
possessionTeam                                                          CIN
playType                                                     play_type_pass
yardlineSide                                                            CIN
yardlineNumber                                                           37
offenseFormation                                                    SHOTGUN
personnelO                                                 1 RB, 1 TE, 3 WR
defendersInTheBox                                                         6
numberOfPass

In [13]:
game.plays[0].player_tracking

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
82744,2018-09-09T17:03:59.099Z,41.95,25.05,0.00,0.00,0.01,121.72,159.17,None,2495143.0,Andy Dalton,14.0,QB,1,away,2018090902,81,right,NaN
82746,2018-09-09T17:03:59.099Z,44.74,5.91,0.00,0.00,0.01,101.14,266.12,None,2495450.0,A.J. Green,18.0,WR,1,away,2018090902,81,right,GO
82758,2018-09-09T17:03:59.099Z,60.93,15.63,0.76,0.95,0.07,264.50,177.76,None,2552386.0,Clayton Geathers,26.0,FS,1,home,2018090902,81,right,NaN
82760,2018-09-09T17:03:59.099Z,45.48,20.02,0.00,0.00,0.01,44.07,341.52,None,2552559.0,C.J. Uzomah,87.0,TE,1,away,2018090902,81,right,NaN
82765,2018-09-09T17:03:59.099Z,44.33,35.09,0.04,0.04,0.07,33.46,269.29,None,2555466.0,Tyler Boyd,83.0,WR,1,away,2018090902,81,right,CROSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85357,2018-09-09T17:04:09.099Z,29.66,28.10,8.59,1.58,0.86,302.59,301.73,None,2558108.0,Nate Hairston,27.0,CB,101,home,2018090902,81,right,NaN
85358,2018-09-09T17:04:09.099Z,16.78,49.67,2.64,4.60,0.29,218.54,310.64,None,2558976.0,Kenny Moore,23.0,CB,101,home,2018090902,81,right,NaN
85359,2018-09-09T17:04:09.099Z,21.60,46.53,8.18,1.46,0.82,277.40,281.52,None,2560789.0,Darius Leonard,53.0,LB,101,home,2018090902,81,right,NaN
85360,2018-09-09T17:04:09.099Z,22.85,48.22,7.01,2.93,0.72,269.89,276.77,None,2560816.0,Skai Moore,55.0,ILB,101,home,2018090902,81,right,NaN
